In [1]:
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [6]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAw1_JIzpuVvwteJCma0xMla?dl=0

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk import word_tokenize


ModuleNotFoundError: No module named 'nltk'

In [8]:
filter=pd.read_csv(r'/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0',sep=',', encoding='latin-1')

FileNotFoundError: [Errno 2] No such file or directory: '/content/AAAw1_JIzpuVvwteJCma0xMla?dl=0'

In [9]:
filter

filter

In [10]:
filter.isnull().sum()

AttributeError: type object 'filter' has no attribute 'isnull'

In [11]:
for col in ['question_text','target']:
  print(col,',',filter[col].value_counts())

TypeError: 'type' object is not subscriptable

In [ ]:
filter.drop([1306122,1306123],0,inplace=True)

In [ ]:
filter.isnull().sum()

In [ ]:
filter.drop(['PK'],1,inplace=True)

In [13]:
filter.dtypes

question_text     object
target           float64
dtype: object

In [14]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model

from nltk import word_tokenize

from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

In [15]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2020-11-04 06:53:49--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2020-11-04 06:53:49--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2020-11-04 06:53:50--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [16]:
!unzip /content/glove.42B.300d.zip 

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [17]:
!rm /content/glove.42B.300d.zip

In [18]:
embeding_index={}

f=open('glove.42B.300d.txt',encoding='utf-8')

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float32')
    embeding_index[word]=coefs
f.close()

In [19]:
len(embeding_index['provide'])

300

In [20]:
embeding_index['provide']

array([ 2.1902e-01, -6.6693e-02,  1.0538e-01, -2.0213e-01,  3.4797e-01,
       -8.4570e-03, -4.0350e+00,  3.4057e-01,  1.0484e-01,  2.1855e-01,
       -4.0837e-01,  2.7482e-01,  1.1953e-01,  7.4335e-02, -1.7069e-01,
       -4.0008e-01, -7.9120e-03, -3.9702e-01,  5.1159e-02, -2.3763e-02,
       -4.6144e-01,  4.7578e-02, -1.8110e-01,  3.9715e-03,  1.5718e-01,
       -1.8605e-01,  3.4680e-01,  1.7597e-01, -5.9390e-02, -3.0119e-01,
       -2.8326e-01, -2.6415e-01, -1.9358e-02,  6.7555e-03,  3.3796e-01,
       -3.5508e-01,  1.0336e-02, -5.0465e-01,  1.3932e-01, -2.7452e-01,
       -1.6377e-01,  5.0877e-01, -1.4303e-01, -2.0842e-01,  2.0463e-01,
        7.3660e-02, -1.8813e-01,  2.8356e-01, -2.7095e-01, -2.5136e-01,
       -1.2715e-01,  2.1164e-01, -4.4701e-01,  9.5515e-02,  4.5539e-01,
       -6.0036e-02,  1.4488e-02, -2.9433e-01, -5.5977e-01,  2.8814e-01,
        9.5750e-03,  2.2739e-01, -8.2802e-02,  3.0510e-01,  3.8501e-02,
        3.2902e-01, -1.1190e-02, -1.2909e-01,  3.6758e-01, -2.29

In [21]:
len(embeding_index)

1917494

In [22]:
X=filter['question_text']
Y=filter['target']

In [23]:
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1044897,), (261225,), (1044897,), (261225,))

In [24]:
class_weight=class_weight.compute_class_weight('balanced',np.unique(y_train),y_train)
print(class_weight)

[0.53292908 8.09207286]


In [25]:
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

{0: 0.5329290833532245, 1: 8.092072859067887}

In [26]:
x_train

904384     Is it possible for diplomats to not have diplo...
897212     How do people in the suburbs differ from peopl...
889934     What does it mean to fight with demons in a dr...
4988                                  Is porn good for kids?
436220                      What will make bitcoin collapse?
                                 ...                        
400116     Is it safe to visit South Korea for a business...
482758     Is there a hormone fluctuation at the end lf t...
1235739    Can you give me a recipe of a low-calorie, hig...
81862      If a company is marketing and selling a softwa...
726565     Is there any problem if I stay single for the ...
Name: question_text, Length: 1044897, dtype: object

In [27]:
from nltk import word_tokenize

In [28]:
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [29]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

361


In [30]:
np.percentile(sent_lens,99)

44.0

In [31]:
max_len=44

tk=Tokenizer(char_level=False,split=' ')

tk.fit_on_texts(x_train)

seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)

vocab_size=len(tk.word_index)

seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)

In [32]:
tk.word_index

{'the': 1,
 'what': 2,
 'is': 3,
 'a': 4,
 'to': 5,
 'in': 6,
 'of': 7,
 'i': 8,
 'how': 9,
 'and': 10,
 'do': 11,
 'are': 12,
 'for': 13,
 'you': 14,
 'can': 15,
 'why': 16,
 'it': 17,
 'my': 18,
 'that': 19,
 'if': 20,
 'with': 21,
 'or': 22,
 'on': 23,
 'have': 24,
 'be': 25,
 'does': 26,
 'from': 27,
 'your': 28,
 'an': 29,
 'which': 30,
 'should': 31,
 'when': 32,
 'get': 33,
 'best': 34,
 'would': 35,
 'as': 36,
 'people': 37,
 'some': 38,
 'there': 39,
 'who': 40,
 'will': 41,
 'like': 42,
 'at': 43,
 'not': 44,
 'about': 45,
 'by': 46,
 'they': 47,
 'did': 48,
 'was': 49,
 'any': 50,
 'we': 51,
 'so': 52,
 'good': 53,
 'me': 54,
 'their': 55,
 'one': 56,
 'india': 57,
 'has': 58,
 'after': 59,
 'most': 60,
 'where': 61,
 'make': 62,
 'this': 63,
 'but': 64,
 'more': 65,
 'all': 66,
 'think': 67,
 'many': 68,
 'between': 69,
 'time': 70,
 'than': 71,
 'much': 72,
 'other': 73,
 'life': 74,
 'someone': 75,
 'use': 76,
 'he': 77,
 'out': 78,
 'way': 79,
 'am': 80,
 'know': 81,
 'u

In [33]:
vocab_size

196123

In [34]:
seq_train_matrix.shape

(1044897, 44)

In [35]:
seq_train

[[3, 17, 109, 13, 12016, 5, 44, 24, 5494, 7669, 149, 1, 88272, 484],
 [9, 11, 37, 6, 1, 13236, 1335, 27, 37, 6, 3275, 1395],
 [2, 26, 17, 116, 5, 572, 21, 7711, 6, 4, 746],
 [3, 1139, 53, 13, 528],
 [2, 41, 62, 914, 3454],
 [20,
  8,
  21239,
  1,
  325,
  267,
  1922,
  5,
  75,
  3208,
  10,
  1747,
  17,
  4,
  14771,
  2317,
  10,
  2069,
  8773,
  7620,
  43,
  1,
  2367,
  10,
  328,
  35,
  75,
  6,
  1,
  11133,
  24,
  5,
  274,
  196,
  1,
  895,
  184],
 [12, 55, 50, 53, 2219, 1387, 23, 104],
 [26, 1, 2839, 2479, 915, 151, 6, 934, 41923, 3011],
 [11, 14, 1334, 1, 2243, 13608],
 [2, 12, 24998, 10701, 10, 1602, 11, 47, 1154],
 [9, 3, 1, 1330, 2840, 1585, 5495, 46, 73, 1509],
 [9, 11, 8, 740, 88273, 27, 3609, 3493],
 [9, 26, 415, 2981, 33, 2856, 13, 6273, 1415],
 [15, 14, 553, 28, 2423, 46, 95, 1014, 17],
 [15, 8350, 33, 6545],
 [20,
  1465,
  278,
  19,
  157,
  1287,
  37,
  3777,
  5333,
  16,
  11,
  47,
  4296,
  19,
  1116,
  5334,
  49,
  4102,
  32,
  77,
  111,
  1656,

In [36]:
x_train.shape

(1044897,)

In [37]:
len(seq_train),len(seq_test)

(1044897, 261225)

In [38]:
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 44), (261225, 44))

In [39]:
embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embeding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector

In [40]:
 len(embedding_matrix)

196124

In [41]:
embedding_matrix.shape

(196124, 300)

In [42]:
embedding_matrix

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [-0.20838   , -0.14932001, -0.017528  , ..., -0.54066002,
         0.21199   , -0.0094357 ],
       [ 0.13877   , -0.0021201 , -0.091039  , ...,  0.024614  ,
         0.46514001,  0.11363   ],
       ...,
       [-0.36572999,  0.35183999,  0.044856  , ...,  0.1814    ,
         0.13234   , -0.11295   ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [43]:
inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)

In [44]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      [(None, 44)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 44, 300)           58837200  
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1665024   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

In [45]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)

In [46]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [47]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [49]:
model.fit(seq_train_matrix,y_train,
              epochs=10,
              class_weight={0: 0.5331085383324966, 1: 8.050922287457816},
              batch_size=1000,
              validation_data=(seq_test_matrix,y_test),
              callbacks = [earlystop, checkpoint])

Epoch 1/10
1045/1045 [==============================] - ETA: 0s - loss: 0.2964 - accuracy: 0.8751
Epoch 00001: val_loss improved from inf to 0.26759, saving model to /content/weights-01-0.2676.h5
1045/1045 [==============================] - 415s 398ms/step - loss: 0.2964 - accuracy: 0.8751 - val_loss: 0.2676 - val_accuracy: 0.8832
Epoch 2/10
1045/1045 [==============================] - ETA: 0s - loss: 0.2508 - accuracy: 0.8951
Epoch 00002: val_loss improved from 0.26759 to 0.23016, saving model to /content/weights-02-0.2302.h5
1045/1045 [==============================] - 413s 396ms/step - loss: 0.2508 - accuracy: 0.8951 - val_loss: 0.2302 - val_accuracy: 0.9088
Epoch 3/10
1045/1045 [==============================] - ETA: 0s - loss: 0.2273 - accuracy: 0.9034
Epoch 00003: val_loss improved from 0.23016 to 0.22493, saving model to /content/weights-03-0.2249.h5
1045/1045 [==============================] - 413s 395ms/step - loss: 0.2273 - accuracy: 0.9034 - val_loss: 0.2249 - val_accuracy: 

In [51]:
best_model = load_model('/content/weights-04-0.1858.h5')

In [52]:
best_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      [(None, 44)]              0         
_________________________________________________________________
embedding (Embedding)        (None, 44, 300)           58837200  
_________________________________________________________________
lstm (LSTM)                  (None, 512)               1665024   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 256)               131328    
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)              

In [53]:
p = best_model.predict(seq_test_matrix)
roc_auc_score(y_test,p)

0.9652133366068854

In [54]:
best_model.save('./spam_filter_model_quora_questions.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>